In [ ]:
demo to using pyspark in OCI by use of DATA FLOW STUDIO

In [ ]:
import json


def prepare_command(command: dict) -> str:
    """Converts dictionary command to the string formatted commands."""
    return f"'{json.dumps(command)}'"

Authenticate with ADS

In [ ]:
import ads

ads.set_auth("resource_principal")  # Supported values: resource_principal, api_key

In [ ]:
import os

compartment_id = os.environ.get("NB_SESSION_COMPARTMENT_OCID")
logs_bucket_uri = "oci://<bucket_name>@<namespace>/<prefix>"
metastore_id = "<metastore_id>"

Dataflow spark magic allows for interaction with dataflow spark clusters

In [ ]:
%load_ext dataflow.magics

In [ ]:
%help

Create Session

In [ ]:
command = prepare_command(
    {
        "compartmentId": compartment_id,
        "displayName": "TestDataFLowSessionFlexShapes",
        "language": "PYTHON",
        "sparkVersion": "3.2.1",
        "numExecutors": 1,
        "driverShape": "VM.Standard.E4.Flex",
        "executorShape": "VM.Standard.E4.Flex",
        "driverShapeConfig": {"ocpus": 2, "memoryInGBs": 32},
        "executorShapeConfig": {"ocpus": 2, "memoryInGBs": 32},
        "type": "SESSION",
        "logsBucketUri": logs_bucket_uri,
        "configuration": {
            "fs.oci.client.hostname": "https://objectstorage.us-ashburn-1.oraclecloud.com"
        },
    }
)

%create_session -l python -c $command

Autoscaling

In [ ]:
command = prepare_command(
    {
        "compartmentId": compartment_id,
        "displayName": "TestDataFLowSessionDynamicAllocation",
        "language": "PYTHON",
        "sparkVersion": "3.2.1",
        "numExecutors": 2,
        "driverShape": "VM.Standard.E4.Flex",
        "executorShape": "VM.Standard.E4.Flex",
        "driverShapeConfig": {"ocpus": 2, "memoryInGBs": 32},
        "executorShapeConfig": {"ocpus": 2, "memoryInGBs": 32},
        "configuration": {
            "fs.oci.client.hostname": "https://objectstorage.us-ashburn-1.oraclecloud.com",
            "spark.dynamicAllocation.enabled": "true",
            "spark.dynamicAllocation.shuffleTracking.enabled": "true",
            "spark.dynamicAllocation.minExecutors": "1",
            "spark.dynamicAllocation.maxExecutors": "4",
            "spark.dynamicAllocation.executorIdleTimeout": "60",
            "spark.dynamicAllocation.schedulerBacklogTimeout": "60",
            "spark.dataflow.dynamicAllocation.quotaPolicy": "min",
        },
    }
)


Configure session

In [ ]:
command = prepare_command(
    {
        "driverShape": "VM.Standard.E4.Flex",
        "executorShape": "VM.Standard.E4.Flex",
        "driverShapeConfig": {"ocpus": 2, "memoryInGBs": 32},
        "executorShapeConfig": {"ocpus": 2, "memoryInGBs": 32},
        "numExecutors": 2,
        "configuration": {
            "fs.oci.client.hostname": "https://objectstorage.us-ashburn-1.oraclecloud.com"
        },
    }
)

Activate session

In [ ]:
command = prepare_command(
    {
        "compartmentId": "<compartment_id>",
        "displayName": "<display_name>",
        "applicationId": "<application_id>",
    }
)


Pyspark

In [ ]:
%%spark
df_nyc_tlc = spark.read.parquet("oci://hosted-ds-datasets@bigdatadatasciencelarge/nyc_tlc/201[1,2,3,4,5,6,7,8]/**/data.parquet", header=False, inferSchema=True)
df_nyc_tlc.show()

df_nyc_tlc.createOrReplaceTempView("nyc_tlc")

In [ ]:
%%spark -c sql -o df_nyc_tlc
SELECT vendor_id, passenger_count, trip_distance, payment_type FROM nyc_tlc LIMIT 1000;

In [ ]:
type(df_nyc_tlc)

Clean up

In [ ]:

%stop_session